In [51]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import json

In [2]:
df = pd.read_csv('labelled_obi.csv')
df = df.iloc[:, 1:]
df.head()

,tweet,sentiment,score,location,hashtags
0,@user @user @user @user @user ....he invested ...,neutral,0.549163,"Lagos, Nigeria",NaN
1,@user @user @user,neutral,0.610686,NaN,NaN
2,@user God bless you for taking time to spell t...,positive,0.893050,"London, England",peterisbetter PeterObi4President2023 ObiDatti2023
3,@user @user AMEN. Happy birthday sir. Alhaji A...,positive,0.973588,NaN,NaN
4,@user @user @user Nwoke okpontu...,neutral,0.801305,ana igbo,NaN


In [3]:
df = df[['sentiment', 'location']]
df.head()

,sentiment,location
0,neutral,"Lagos, Nigeria"
1,neutral,NaN
2,positive,"London, England"
3,positive,NaN
4,neutral,ana igbo


In [81]:
neg_df = df[df['sentiment'] == 'negative'].sample()
neu_df = df[df['sentiment'] == 'neutral'].sample()
pos_df = df[df['sentiment'] == 'positive'].sample()

# test = [neu_df, neg_df, pos_df]
result = pd.concat([neu_df, neg_df, pos_df])

# print(test)
result

,sentiment,location
4704,neutral,"Toronto, Ontario"
6161,negative,We Global.
1604,positive,"Cairo, Egypt"


In [79]:
result.to_json(orient='index')

'{"9072":{"sentiment":"neutral","location":null},"1449":{"sentiment":"negative","location":"Cape coast Ghana "},"2880":{"sentiment":"positive","location":null}}'

In [72]:
neg_df

,sentiment,location
1503,negative,"Kaduna, Nigeria"


In [6]:
neu_df

,sentiment,location
0,neutral,"Lagos, Nigeria"
1,neutral,NaN
4,neutral,ana igbo
6,neutral,"Lagos, Nigeria"
10,neutral,NaN
...,...,...
9989,neutral,"Abuja, Nigeria"
9990,neutral,"Maicon, Georgia"
9992,neutral,NaN
9999,neutral,"Lagos, Nigeria"


In [7]:
pos_df

,sentiment,location
2,positive,"London, England"
3,positive,NaN
8,positive,NaN
9,positive,Billionaire’s quaters
21,positive,NaN
...,...,...
9963,positive,NaN
9966,positive,NaN
9969,positive,"Montréal, Québec"
9974,positive,Nigeria


In [8]:
neg_df = neg_df.dropna()
neg_df['location'] = neg_df['location'].str.lower()
neg_df[neg_df.location.str.contains('niger')]

C:\Users\USER\AppData\Local\Temp\ipykernel_7896\16740512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_df['location'] = neg_df['location'].str.lower()


,sentiment,location
5,negative,"lagos, nigeria"
7,negative,"port harcourt, nigeria"
29,negative,"lagos, nigeria"
45,negative,"lagos, nigeria"
64,negative,"lagos, nigeria"
...,...,...
9968,negative,"malumfashi, nigeria"
9975,negative,"ikeja, nigeria"
9977,negative,"ikeja, nigeria"
9981,negative,"port harcourt, nigeria"


In [9]:
neg_location = list(neg_df['location'])
neg_location = [str(i) for i in neg_location]
neg_location = [i.lower() for i in neg_location]

In [10]:
states = [
        'abia', 'adamawa', 'akwa ibom', 'anambra', 'bauchi', 'bayelsa',
        'benue', 'borno', 'cross river', 'delta', 'ebonyi', 'edo',
        'ekiti', 'enugu', 'gombe', 'imo', 'jigawa', 'kaduna',
        'kano', 'katsina', 'kebbi', 'kogi', 'kwara', 'lagos',
        'nasarawa', 'niger', 'ogun', 'ondo', 'osun', 'oyo',
        'plateau', 'rivers', 'sokoto', 'taraba', 'yobe', 'zamfara'
]

In [11]:
neg_location_dict = {}
for state in states:
    neg_test = {state: [j for j in neg_location if state in j]}
    neg_location_dict.update(neg_test)

In [12]:
neg_location_dict

{'abia': ['abia, nigeria', 'abia, nigeria', 'abia'],
 'adamawa': ['adamawa state '],
 'akwa ibom': ['akwa ibom, nigeria',
  'uyo, akwa ibom ',
  'akwa ibom, nigeria',
  'akwa ibom, nigeria',
  'uyo,akwa ibom state. nigeria',
  'akwa ibom, nigeria',
  'uyo, akwa ibom '],
 'anambra': ['anambra',
  'anambra, nigeria',
  'anambra, nigeria',
  'anambra, nigeria',
  'anambra, nigeria',
  'anambra, nigeria',
  'anambra, nigeria',
  'anambra, nigeria',
  'anambra, nigeria',
  'anambra, nigeria'],
 'bauchi': ['bauchi, nigeria', 'bauchi', 'bauchi, nigeria', 'bauchi, nigeria'],
 'bayelsa': ['bayelsa, nigeria'],
 'benue': ['benue, nigeria', 'benue, nigeria'],
 'borno': ['borno state ', 'borno, nigeria', 'borno, nigeria', 'borno/biu'],
 'cross river': ['cross river',
  'cross river, nigeria',
  'cross river',
  'cross river',
  'cross river',
  'cross river',
  'cross river'],
 'delta': ['delta state, nigeria',
  'delta state, nigeria',
  'delta state, nigeria',
  'asaba, delta, nigeria',
  'delta,

In [38]:
test = [s for s in neg_location if re.match(r'\bniger\b', s, flags=re.IGNORECASE)]
print(test)

[]


In [56]:
pattern = r"\b(abia|abuja|adamawa|akwa ibom|anambra|bauchi|bayelsa|benue|borno|cross river|delta|ebonyi|edo|ekiti|enugu|gombe|imo|jigawa|kaduna|kano|katsina|kebbi|kogi|kwara|lagos|nasarawa|niger|ogun|ondo|osun|oyo|plateau|rivers|sokoto|taraba|yobe|zamfara)\b"

counts = {}
for item in neg_location:
    matches = re.findall(pattern, item, flags=re.IGNORECASE)
    for match in matches:
        key = match.lower()
        if key in counts:
            counts[key] += 1
        else:
            counts[key] = 1

print(counts)

{'lagos': 484, 'imo': 8, 'abuja': 135, 'cross river': 7, 'akwa ibom': 7, 'bauchi': 4, 'delta': 17, 'enugu': 28, 'ogun': 15, 'osun': 1, 'rivers': 11, 'borno': 4, 'kaduna': 8, 'gombe': 3, 'adamawa': 1, 'kebbi': 1, 'niger': 3, 'ebonyi': 2, 'anambra': 10, 'katsina': 1, 'bayelsa': 1, 'jigawa': 2, 'kano': 4, 'edo': 1, 'benue': 2, 'abia': 3, 'ekiti': 2, 'ondo': 2, 'plateau': 5, 'sokoto': 1, 'kogi': 1, 'zamfara': 1, 'taraba': 1}


In [62]:
count_df = pd.DataFrame.from_dict(counts, orient='index')
count_df.columns = ['location_count']
count_df

,location_count
lagos,484
imo,8
abuja,135
cross river,7
akwa ibom,7
bauchi,4
delta,17
enugu,28
ogun,15
osun,1


In [68]:
count_df.sample()

,location_count
abia,3
